In [1]:
import os
from dotenv import load_dotenv, find_dotenv 

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "ChatbotMemory"

In [2]:
from langchain_openai import ChatOpenAI 

In [3]:
model = ChatOpenAI(model = "gpt-3.5-turbo")

In [4]:
from langchain_core.output_parsers import StrOutputParser 
parser = StrOutputParser() 

In [5]:
from langchain_core.messages import HumanMessage

In [6]:
# while True:
#     message = input("Enter a query")
#     if message == "Goodbye":
#         print("Have a great day")
#         break
#     else:
#         print(parser.invoke(model.invoke([HumanMessage(content=message)])))

In [7]:
from langchain_core.chat_history import BaseChatMessageHistory 
from langchain_core.chat_history import InMemoryChatMessageHistory 
from langchain_core.runnables.history import RunnableWithMessageHistory

In [8]:
store = {}

In [9]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory() 
    return store[session_id]

In [10]:
config = {"configurable": {"session_id": "firstchat"}}

In [11]:
model_with_memory = RunnableWithMessageHistory(model, get_session_history)

In [12]:
model_with_memory.invoke([HumanMessage(content="Hi, I'm Hitesh")], config = config).content

'Hello Hitesh, nice to meet you! How can I assist you today?'

In [13]:
model_with_memory.invoke([HumanMessage(content="Whats my name ?")], config=config).content

'Your name is Hitesh.'

In [14]:
config = {"configurable": {"session_id": "secondchat"}}

In [15]:
model_with_memory.invoke([HumanMessage(content="What is my name ?")], config = config).content

"I'm sorry, I do not know your name as I am an AI assistant and we have not been properly introduced."

In [16]:
config = {"configurable": {"session_id": "firstchat"}}
model_with_memory.invoke([HumanMessage(content="Whats my name?")], config = config).content

'Your name is Hitesh.'

In [17]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [18]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful assistant, Answer all questions to the best of your ability"),
    MessagesPlaceholder(variable_name="messages")
])

In [19]:
chain = prompt | model | parser

In [20]:
chain.invoke({"messages": [HumanMessage(content="Hi im Hitesh")]})

'Hello Hitesh! How can I assist you today?'

In [21]:
chain.invoke({"messages": [HumanMessage(content="What is my name ?")]})

"I'm sorry, but as an AI assistant, I do not have access to personal information about users. How can I assist you today?"

In [22]:
model_with_memory = RunnableWithMessageHistory(chain, get_session_history)

In [23]:
config = {"configurable": {"session_id" : "third_chat"}}
response = model_with_memory.invoke(
    [HumanMessage(content="Hello im John"), 
     ], config=config
)

In [24]:
response

'Hello John! How can I assist you today?'

In [25]:
response = model_with_memory.invoke(
    [HumanMessage(content="What is my name ?"), 
     ], config=config
)

In [26]:
response

'Your name is John.'

### **Trimming Conversation**

In [27]:
from langchain_core.messages import SystemMessage, trim_messages, HumanMessage, AIMessage

In [28]:
trimmer = trim_messages(
    max_tokens = 40, 
    strategy = "last", 
    token_counter = model, 
    include_system = True, 
    allow_partial = False, 
    start_on = "human",
)

In [29]:
messages = [
    HumanMessage(content="Hi, Im John"), 
    AIMessage(content="hi"), 
    HumanMessage(content="I like eating food"), 
    AIMessage("Enjoy your food"), 
    HumanMessage(content="Whats 2 + 2"), 
    AIMessage(content="4"), 
    HumanMessage(content="Thanks"), 
    AIMessage(content="No Problem"), 
    HumanMessage(content="Are you having fun ?"), 
    AIMessage(content="Yes!")
]

In [30]:
model.get_num_tokens_from_messages(messages)

72

In [31]:
trimmed_message = trimmer.invoke(messages)

In [32]:
model.get_num_tokens_from_messages(trimmed_message)

29

In [33]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful assistant, Answer all questions to the best of your ability"),
    MessagesPlaceholder(variable_name="messages")
])

In [34]:
trimmed_message + [HumanMessage(content="Whats my name ?")]

[HumanMessage(content='Thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='No Problem', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Are you having fun ?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Yes!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Whats my name ?', additional_kwargs={}, response_metadata={})]

In [35]:
from operator import itemgetter 
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages = itemgetter("messages") | trimmer)
    | prompt 
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="Whats my name ?")],
        "language": "Spanish", 
    }
)

In [36]:
response.content

"I'm sorry, I don't have access to personal information like your name."

In [37]:
response = chain.invoke(
    {
        "messages" : messages + [HumanMessage(content="What math problem did i asked ?")], 
        "language" : "English"
    }
)

In [38]:
response.content

"You didn't ask a specific math problem. If you have a math question, feel free to ask!"